# Project目標 

Line Chat BOT on Heroku

# 專案目錄資料
![title](HW30_linechatbot.png)

# README.txt

### Ubuntu上安裝Heroku_CLI
sudo snap install heroku --classic

### requirements.txt 是要叫Heroku透過pip install 安裝的套件名稱
* flask
* gunicorn
* line-bot-sdk

### runtime.txt 是要運行的環境
python-3.7.8

### Procfile 是告訴 Heroku 我的程式該怎麼跑起來 
web: gunicorn linechatbot:app

### 登入Heroku
heroku login

### 在project目錄下執行, 我這邊是以 terry-chatbot 命名, 跟我在Heroku建立的Application是一樣的名字
* git init
* heroku git:remote -a terry-chatbot
* git add -A
* git status
* git rm --cached .vscode/settings.json
* git commit -am "terry-chatbot v2"
* git push heroku master

### 查看目前機器狀況
heroku ps

### 遠端關機
heroku ps:scale web=0

### 遠端開機
heroku ps:scale web=1

### 在Chrome開啟網址連結
heroku open

### 查看後台即時log
heroku logs --tail
> Press Control+C to stop streaming the logs.

# linechatbot.py

In [ ]:
from flask import Flask,request,abort
from linebot import LineBotApi,WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent,TextMessage,TextSendMessage

import configparser


app = Flask(__name__)
config = configparser.ConfigParser()
config.read('config.ini')
#print("channel_token:" + config.get('line-bot', 'channel_token'))
#print("channel_secret:" + config.get('line-bot', 'channel_secret'))
line_bot_api = LineBotApi(config.get('line-bot', 'channel_token'))
handler = WebhookHandler(config.get('line-bot', 'channel_secret'))

@app.route("/")
def main():
    return "Terry Chat Bot Test!!"

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)
    return 'OK'

@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    print("event message:" + event.message.text)
    line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text=event.message.text))

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8081, debug=True)
